In [ ]:
import helpers as hlp
import plotting as aplt
import iris
import control_experiment_1 as experiment_runner
from pathlib import Path
import numpy as np
import proplot as pplt
import xarray as xr

In [ ]:
plotting_output_directory = Path("plots/control_experiment_1")
plotting_output_directory.mkdir(exist_ok=True)

In [ ]:
naive_exp_ids = ["C1N0", "C1N1", "C1N2"]
schwarz_exp_ids = ["C1S0", "C1S1", "C1S2"]
max_schwarz_iters = experiment_runner.max_iters

In [ ]:
def load_variables(setup: str, exp_ids: list):
    oce_t_files = [f"{setup}/{exp_id}/{exp_id[:4]}*_T.nc" for exp_id in exp_ids]
    atm_prog_files = [f"{setup}/{exp_id}/progvar.nc" for exp_id in exp_ids]
    atm_diag_files = [f"{setup}/{exp_id}/diagvar.nc" for exp_id in exp_ids]
    atm_temps = [
        hlp.load_cube(atm_prog_file, "Temperature") for atm_prog_file in atm_prog_files
    ]
    oce_ssts = [
        hlp.load_cube(oce_t_file, "Sea Surface temperature")
        for oce_t_file in oce_t_files
    ]
    atm_ssws = [
        hlp.load_cube(atm_diag_file, "Surface SW Radiation")
        for atm_diag_file in atm_diag_files
    ]
    return atm_temps, oce_ssts, atm_ssws

In [ ]:
setup = "PAPA"
atm_temps, oce_ssts, atm_ssws = load_variables(setup, naive_exp_ids)

colors = ["m", "c", "y"]
labels = ["parallel", "atm-first", "oce-first"]
alpha = 1
linestyles = ["--", ":", "-."]

fig, axs = pplt.subplots(nrows=3, ncols=1)
fig.set_size_inches(15, 10)
fig.suptitle(
    f"{naive_exp_ids[0]}, {naive_exp_ids[1]}, {naive_exp_ids[2]}", y=0.95, size=14
)

aplt.create_atm_temps_plot(axs[0], atm_temps, colors, alpha, labels, linestyles)
aplt.create_oce_ssts_plot(axs[1], oce_ssts, colors, alpha, labels, linestyles)
aplt.create_atm_ssws_plot(axs[2], atm_ssws, colors, alpha, labels, linestyles)
fig.savefig(
    plotting_output_directory / "naive_coupling_schemes.pdf",
    bbox_inches="tight",
)

In [ ]:
setup = "PAPA"
final_iters = [f"{exp_id}_{max_schwarz_iters}" for exp_id in schwarz_exp_ids]
atm_temps, oce_ssts, atm_ssws = load_variables(setup, final_iters)

colors = ["m", "c", "y"]
labels = ["parallel", "atm-first", "oce-first"]
alpha = 0.7
linestyles = ["--", ":", "-."]

fig, axs = pplt.subplots(nrows=3, ncols=1)
fig.set_size_inches(15, 10)
fig.suptitle(f"Final Schwarz Iterations for Control Experiment 1", y=0.95, size=14)

aplt.create_atm_temps_plot(axs[0], atm_temps, colors, alpha, labels, linestyles)
aplt.create_oce_ssts_plot(axs[1], oce_ssts, colors, alpha, labels, linestyles)
aplt.create_atm_ssws_plot(axs[2], atm_ssws, colors, alpha, labels, linestyles)
fig.savefig(
    plotting_output_directory / "schwarz_coupling.pdf",
    bbox_inches="tight",
)

# Naive vs. Schwarz Solution

In [ ]:
setup = "PAPA"
atm_temps, oce_ssts, atm_ssws = load_variables(
    setup, [*naive_exp_ids, f"{schwarz_exp_ids[0]}_{max_schwarz_iters}"]
)

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atmosphere-first", "ocean-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(nrows=3, ncols=1)
fig.set_size_inches(15, 10)
fig.suptitle(f"Non-iterative vs. Schwarz (CE1)", y=0.95, size=14)

aplt.create_atm_temps_plot(axs[0], atm_temps, colors, alpha, labels, linestyles)
aplt.create_oce_ssts_plot(axs[1], oce_ssts, colors, alpha, labels, linestyles)
aplt.create_atm_ssws_plot(axs[2], atm_ssws, colors, alpha, labels, linestyles)
axs[0].legend(ncols=1)
fig.savefig(
    plotting_output_directory / "coupling_comparison.pdf",
    bbox_inches="tight",
)

# Boundary layer type

In [ ]:
atm_diag_files = [f"PAPA/{exp_id}/diagvar.nc" for exp_id in naive_exp_ids]
atm_diag_files.append(f"PAPA/{schwarz_exp_ids[0]}_{max_schwarz_iters}/diagvar.nc")
varname = "pbl_type"
pbl_types = [hlp.load_cube(file, varname) for file in atm_diag_files]

# swr_pbl_type = pbl_types[-1]
# for pbl_type in pbl_types[:-1]:
#     iris.util.mask_cube(pbl_type, (pbl_type.data - swr_pbl_type.data == 0))

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
markers = [".", ".", ".", "."]
shifts = [-0.06, -0.02, 0.02, 0.06]

fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Boundary Layer Type in Control Experiment 1", y=0.95, size=14)

for i in range(len(colors)):
    pbl_type = pbl_types[i]
    color = colors[i]
    label = labels[i]
    marker = markers[i]
    shift = shifts[i]
    time_coord = pbl_type.coord("time")
    new_time_coord = iris.coords.DimCoord(
        time_coord.points,
        standard_name="time",
        long_name="Time",
        var_name="time",
        units="seconds since 2014-07-01 00:00:00",
    )
    # time shift: -7h from UTC to PDT
    new_time_coord.points = new_time_coord.points - 7 * 3600
    pbl_type.remove_coord("time")
    pbl_type.add_dim_coord(new_time_coord, 0)
    da = xr.DataArray.from_iris(pbl_type + shift)
    ax.plot(
        da,
        color=color,
        label=label,
        alpha=alpha,
        ls="none",
        marker=marker,
    )
ax.set_ylabel("Boundary layer type")
ax.set_yticks([0, 1, 2, 3])
ax.set_yticklabels(["DS", "DC", "Sc", "Cu"])
# ax.set_title(r"only shows points where BL type $\neq$ SWR result")
ax.legend(ncol=1)
fig.savefig(
    plotting_output_directory / "pbl_type_comparison.pdf",
    bbox_inches="tight",
)

In [ ]:
atm_diag_files = [f"PAPA/{exp_id}/diagvar.nc" for exp_id in naive_exp_ids]
atm_diag_files.append(f"PAPA/{schwarz_exp_ids[0]}_{max_schwarz_iters}/diagvar.nc")
varname = "Surface Sensible Heat Flux"
sh_fluxes = [hlp.load_cube(file, varname) for file in atm_diag_files]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Surface SH Flux in Control Experiment 1", y=0.95, size=14)

for i in range(len(colors)):
    sh_flux = sh_fluxes[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    time_coord = sh_flux.coord("time")
    new_time_coord = iris.coords.DimCoord(
        time_coord.points,
        standard_name="time",
        long_name="Time",
        var_name="time",
        units="seconds since 2014-07-01 00:00:00",
    )
    # time shift: -7h from UTC to PDT
    new_time_coord.points = new_time_coord.points - 7 * 3600
    sh_flux.remove_coord("time")
    sh_flux.add_dim_coord(new_time_coord, 0)
    da = xr.DataArray.from_iris(sh_flux)
    ax.plot(
        da,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.set_ylabel(r"Surface SH Flux $[W m^{-2}]$")
ax.set_xlabel("")
ax.set_title("")
ax.legend(ncols=1)
fig.savefig(
    plotting_output_directory / "ssh_comparison.pdf",
    bbox_inches="tight",
)

# Boundary Layer Heights

## Atmosphere

In [ ]:
atm_diag_files = [f"PAPA/{exp_id}/diagvar.nc" for exp_id in naive_exp_ids]
atm_diag_files.append(f"PAPA/{schwarz_exp_ids[0]}_{max_schwarz_iters}/diagvar.nc")
varname = "pbl_height"
bl_heights = [hlp.load_cube(file, varname) for file in atm_diag_files]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Boundary Layer Height (Control Experiment 1)", y=0.95, size=14)

for i in range(len(colors)):
    bl_height = bl_heights[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    time_coord = bl_height.coord("time")
    new_time_coord = iris.coords.DimCoord(
        time_coord.points,
        standard_name="time",
        long_name="Time",
        var_name="time",
        units="seconds since 2014-07-01 00:00:00",
    )
    # time shift: -7h from UTC to PDT
    new_time_coord.points = new_time_coord.points - 7 * 3600
    bl_height.remove_coord("time")
    bl_height.add_dim_coord(new_time_coord, 0)
    da = xr.DataArray.from_iris(bl_height)
    ax.plot(
        da,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.set_ylabel(r"Boundary Layer Height $[m]$")
ax.set_xlabel("")
ax.set_title("")
ax.legend(ncols=1)
fig.savefig(
    plotting_output_directory / "pbl_height.pdf",
    bbox_inches="tight",
)

In [ ]:
height = hlp.load_cube(atm_diag_files[-1], "height_f")
time_coord = height.coord("time")
new_time_coord = iris.coords.DimCoord(
    time_coord.points,
    standard_name="time",
    long_name="Time",
    var_name="time",
    units="seconds since 2014-07-01 00:00:00",
)
# time shift: -7h from UTC to PDT
new_time_coord.points = new_time_coord.points - 7 * 3600
height.remove_coord("time")
height.add_dim_coord(new_time_coord, 0)

fig, ax = pplt.subplots()
fig.set_size_inches(8, 5)

da = xr.DataArray.from_iris(height[:, 50])
ax.plot(da)
fig.suptitle("Height at Atmospheric Model Level 50")
ax.set_title("")
fig.savefig(
    plotting_output_directory / "height_L50.pdf",
    bbox_inches="tight",
)

$\Rightarrow$ for a vertical profile of the atmospheric boundary layer, the last 10 atmospheric levels should suffice!

## Ocean

In [ ]:
oce_t_files = [f"PAPA/{exp_id}/{exp_id[:4]}*_T.nc" for exp_id in naive_exp_ids]
oce_t_files.append(
    f"PAPA/{schwarz_exp_ids[0]}_{max_schwarz_iters}/{schwarz_exp_ids[0]}*_T.nc"
)
varname = "somxl010"
somxl010s = [hlp.load_cube(file, varname) for file in oce_t_files]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Mixed Layer Depth (Control Experiment 1)", y=0.95, size=14)

for i in range(len(colors)):
    somxl010 = somxl010s[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    time_coord = somxl010.coord("time")
    # time shift: -7h from UTC to PDT
    time_coord.points = time_coord.points - 7 * 3600
    time_coord.bounds = time_coord.bounds - 7 * 3600
    da = xr.DataArray.from_iris(somxl010[:, 0, 0])
    ax.plot(
        da,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.set_ylim([0, 10])
ax.set_ylabel(r"Mixed Layer Depth $[m]$")
ax.set_xlabel("")
ax.set_title("")
ax.legend(ncols=1)
fig.savefig(
    plotting_output_directory / "mldepth.pdf",
    bbox_inches="tight",
)

According to [this report, p. 18](https://www.drakkar-ocean.eu/publications/reports/orca025-grd100-report-dussin), the first 10 levels should suffice for NEMO as well.

# Vertical Temperature Profile

In [ ]:
atm_prog_files = [f"PAPA/{exp_id}/progvar.nc" for exp_id in naive_exp_ids]
atm_prog_files.append(f"PAPA/{schwarz_exp_ids[0]}_{max_schwarz_iters}/progvar.nc")
varname = "t"
temperatures = [hlp.load_cube(file, varname) for file in atm_prog_files]

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1)
fig.set_size_inches(15, 13)
fig.suptitle(
    f"Vertical Atmospheric Temperature Profiles in BL (Control Experiment 1)",
    y=1,
    size=14,
)

pressure = hlp.load_cube(atm_prog_files[0], "Pressure - full levels")
p_coord = iris.coords.DimCoord(
    pressure[0].data,
    standard_name="air_pressure",
    long_name="Pressure - full levels",
    units="Pa",
)

for i in range(len(atm_prog_files)):
    ax = axs[i]
    ax_title = titles[i]
    temperature = temperatures[i]
    temperature.convert_units("degC")
    temperature.add_aux_coord(p_coord, 1)
    temperature.coord("air_pressure").convert_units("hPa")
    time_coord = temperature.coord("time")
    new_time_coord = iris.coords.DimCoord(
        time_coord.points,
        standard_name="time",
        long_name="Time",
        var_name="time",
        units="seconds since 2014-07-01 00:00:00",
    )
    # time shift: -7h from UTC to PDT
    new_time_coord.points = new_time_coord.points - 7 * 3600
    temperature.remove_coord("time")
    temperature.add_dim_coord(new_time_coord, 0)
    da = xr.DataArray.from_iris(temperature)
    da = da.swap_dims({"nlev": "air_pressure"})
    im = ax.contourf(
        da[:, 50:],
        levels=10,
        vmin=0,
        vmax=15,
        transpose=True,
        discrete=True,
    )
    ax.set_ylabel("Air Pressure [hPa]")
    ax.invert_yaxis()
    ax.set_title(ax_title)
fig.colorbar(im, loc="b")
fig.savefig(
    plotting_output_directory / "air_temperature_profile.pdf",
    bbox_inches="tight",
)

In [ ]:
oce_t_files = [f"PAPA/{exp_id}/{exp_id[:4]}*_T.nc" for exp_id in naive_exp_ids]
oce_t_files.append(
    f"PAPA/{schwarz_exp_ids[0]}_{max_schwarz_iters}/{schwarz_exp_ids[0]}*_T.nc"
)
varname = "votemper"
temperatures = [hlp.load_cube(file, varname) for file in oce_t_files]

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1)
fig.set_size_inches(15, 13)
fig.suptitle(
    f"Vertical Ocean Temperature Profiles in Mixed Layer (Control Experiment 1)",
    y=1,
    size=14,
)

for i in range(len(oce_t_files)):
    ax = axs[i]
    ax_title = titles[i]
    temperature = temperatures[i]
    time_coord = temperature.coord("time")
    time_coord.points = time_coord.points - 7 * 3600
    time_coord.bounds = time_coord.bounds - 7 * 3600
    da = xr.DataArray.from_iris(temperature[:, :10, 0, 0])
    im = ax.contourf(
        da,
        levels=10,
        vmin=10,
        vmax=12,
        discrete=True,
        transpose=True,
    )
    ax.invert_yaxis()
    ax.set_title(ax_title)
cax = axs[-1]
fig.colorbar(im, loc="b")
fig.savefig(
    plotting_output_directory / "oce_temperature_profiles.pdf",
    bbox_inches="tight",
)

# Cloud Cover

In [ ]:
atm_diag_files = [f"PAPA/{exp_id}/diagvar.nc" for exp_id in naive_exp_ids]
atm_diag_files.append(f"PAPA/{schwarz_exp_ids[0]}_{max_schwarz_iters}/diagvar.nc")
varname = "total_cloud"
tccovers = [hlp.load_cube(file, varname) for file in atm_diag_files]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Total Cloud Cover (Control Experiment 1)", y=0.95, size=14)

for i in range(len(colors)):
    tccover = tccovers[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    time_coord = tccover.coord("time")
    new_time_coord = iris.coords.DimCoord(
        time_coord.points,
        standard_name="time",
        long_name="Time",
        var_name="time",
        units="seconds since 2014-07-01 00:00:00",
    )
    # time shift: -7h from UTC to PDT
    new_time_coord.points = new_time_coord.points - 7 * 3600
    tccover.remove_coord("time")
    tccover.add_dim_coord(new_time_coord, 0)
    da = xr.DataArray.from_iris(tccover)
    ax.plot(
        da,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.set_ylabel(r"Cloud Cover")
ax.set_title("")
ax.legend(ncols=1)
fig.savefig(
    plotting_output_directory / "ccover.pdf",
    bbox_inches="tight",
)

# Boundary Layer Type across Iterations

How often does the PBL type change for a particular time step compared to the previous iteration?

In [ ]:
atm_diag_files = []
for iter in range(1, max_schwarz_iters + 1):
    atm_diag_files.append(f"PAPA/{schwarz_exp_ids[0]}_{iter}/diagvar.nc")
varname = "pbl_type"
pbl_types = [hlp.load_cube(file, varname) for file in atm_diag_files]

pbl_type_changes = pbl_types[0].copy()
pbl_type_changes.data[:] = 0

previous_pbl_type = pbl_types[0]
for iter in range(1, max_schwarz_iters):
    pbl_type = pbl_types[iter]
    pbl_type_not_equal = np.where(pbl_type.data != previous_pbl_type.data)
    pbl_type_changes.data[pbl_type_not_equal] += 1
    previous_pbl_type = pbl_type

fig, ax = pplt.subplots()
fig.set_size_inches(8, 5)
fig.suptitle(f"Boundary Layer Type Change Counts (CE1)", size=14)
time_coord = pbl_type_changes.coord("time")
new_time_coord = iris.coords.DimCoord(
    time_coord.points,
    standard_name="time",
    long_name="Time",
    var_name="time",
    units="seconds since 2014-07-01 00:00:00",
)
# time shift: -7h from UTC to PDT
new_time_coord.points = new_time_coord.points - 7 * 3600
pbl_type_changes.remove_coord("time")
pbl_type_changes.add_dim_coord(new_time_coord, 0)
da = xr.DataArray.from_iris(pbl_type_changes)
ax.plot(
    da,
    color="k",
    alpha=1,
    ls="none",
    marker=".",
)
ax.set_ylabel("# of PBL type changes")
ax.set_title("+1 if boundary layer type changed between subsequent iterations")
fig.savefig(
    plotting_output_directory / "pbl_type_change_counter.pdf",
    bbox_inches="tight",
)

For the points which change the most frequently: Plot `PBL_Type` vs. iteration

In [ ]:
threshold = 3
frequent_changer_indices = (pbl_type_changes.data > threshold).nonzero()[0]
fig, ax = pplt.subplots()
fig.set_size_inches(w=8, h=5)
x_data = np.arange(1, max_schwarz_iters + 1)
for frequent_changer_index in frequent_changer_indices:
    pbl_type_per_iteration = np.zeros(max_schwarz_iters)
    for iter in range(max_schwarz_iters):
        data = pbl_types[iter].data[frequent_changer_index]
        pbl_type_per_iteration[iter] = data
    _ = ax.plot(
        x_data,
        pbl_type_per_iteration,
        label=frequent_changer_index,
        ls="none",
        marker=".",
        alpha=1,
    )
ax.set_yticks([0, 1, 2, 3])
ax.set_yticklabels(["DS", "DC", "Sc", "Cu"])
ax.set_xticks(x_data[::2])
ax.set_xlabel("Schwarz Iteration")
ax.set_ylabel("Boundary Layer Type")
fig.suptitle(f"BL Type over iterations for points with > {threshold} changes")